<a href="https://colab.research.google.com/github/Alexxxa016/AI/blob/main/SearchTreeCheckrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [212]:
import copy

def in_bounds(r, c):
    return 0 <= r < 8 and 0 <= c < 8

def deep_copy_board(board):
    return copy.deepcopy(board)

In [213]:
def empty_board():
    board = []
    for row in range(8):
        board.append(["."] * 8)
    return board

def print_board(board):
    # Column labels a-h
    col_labels = "   " + " ".join(chr(c) for c in range(ord('a'), ord('h')+1))
    print(col_labels)
    for row in range(8):
        print(f"{row}  " + " ".join(board[row]))
    print()

board = empty_board()
print_board(board)


   a b c d e f g h
0  . . . . . . . .
1  . . . . . . . .
2  . . . . . . . .
3  . . . . . . . .
4  . . . . . . . .
5  . . . . . . . .
6  . . . . . . . .
7  . . . . . . . .



In [214]:
def checkers_board():
    board = []
    for row in range(8):
        row_list = []
        for col in range(8):
            if (row + col) % 2 == 1:
                row_list.append(".")
            else:
                row_list.append(" ")
        board.append(row_list)
    return board
#8x8 checkers board with playable dark squares (".") and non-playable light squares (" ").
def print_board(board):
    col_labels = "   " + " ".join(chr(c) for c in range(ord('a'), ord('h')+1))
    print(col_labels)
    for row in range(8):
        print(f"{row}  " + " ".join(board[row]))
    print()

board = checkers_board()
print_board(board)


   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  .   .   .   .  
4    .   .   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  



In [215]:
# Convert a string like "a3" to board coordinates (row, col)
def parse_position(pos):

  pos = pos.strip().lower()
  rowpos = ""
  colpos = ""

  for char in pos:
    if char.isdigit():
      rowpos += char
    elif char.isalpha():
      colpos += char
  if rowpos == "" or colpos == "":
    raise ValueError("Invalid position format")
  row = int(rowpos)
  col = ord(colpos[0]) - ord('a')
  return row, col




In [216]:
print(parse_position("2f"))  #(2, 5)
print(parse_position("4c"))  #(4, 2)
print(parse_position("6h"))  #(6, 7)
print(parse_position("4d"))  #(4, 3)
print(parse_position("0a"))  #(0, 0)
print(parse_position("7b"))  #(7, 1)

(2, 5)
(4, 2)
(6, 7)
(4, 3)
(0, 0)
(7, 1)


In [217]:
#board state and history of moves to track repeating positions
class CurrentBoard:
    def __init__(self, board=None, history=None):
        if board is None:
            self.board = self.end_game_board()
        else:
            self.board = board
        if history is None:
            self.history = []
        else:
            self.history = history
#endgame board setup
    def end_game_board(self):
        board = []
        for row in range(8):
            row_list = []
            for col in range(8):
                if (row + col) % 2 == 1:
                    row_list.append(".")
                else:
                    row_list.append(" ")
            board.append(row_list)
        board[3][0] = "W"
        board[2][5] = "b"
        board[4][3] = "B"
        return board

    def display(self):
        col_labels = "   " + " ".join(chr(c) for c in range(ord('a'), ord('h') + 1))
        print(col_labels)
        for row in range(8):
            print(f"{row}  " + " ".join(self.board[row]))
        print()

    def state_of_board(self):
        white_moves = len(self.all_moves("w"))
        black_moves = len(self.all_moves("b"))
        if white_moves == 0:
            return "black"
        if black_moves == 0:
            return "white"
        return "Unfinished"

    def opponent(self, piece):
        return "b" if piece.lower() == "w" else "w"
#piece movement
    def piece_moves(self, piece):
        if piece in ("B", "W"):
            return [(-1, -1), (-1, 1), (1, -1), (1, 1)]
        elif piece == "b":
            return [(1, -1), (1, 1)]
        elif piece == "w":
            return [(-1, -1), (-1, 1)]
        return []


In [218]:
cb = CurrentBoard()
cb.display()

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  W   .   .   .  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  



In [219]:
# This function generates all legal moves for the piece at position (r, c).
# It checks normal moves and capture moves. If a capture is possible, that move is prioritized
def generate_moves(self, r, c):
    piece = self.board[r][c]
    if piece not in ("b", "B", "w", "W"):
        return []
    moves = []
    captures = []
    directions = self.piece_moves(piece)
    opp = self.opponent(piece)
    for dr, dc in directions:
        new_r, new_c = r + dr, c + dc
        if in_bounds(new_r, new_c):
            if self.board[new_r][new_c] == ".":
                moves.append((new_r, new_c, None))
            else:
                target = self.board[new_r][new_c]
                if target.lower() == opp:
                    cap_r, cap_c = r + 2*dr, c + 2*dc
                    if in_bounds(cap_r, cap_c) and self.board[cap_r][cap_c] == ".":
                        captures.append((cap_r, cap_c, (new_r, new_c)))
    return captures if captures else moves

CurrentBoard.generate_moves = generate_moves


In [220]:
# returns a list of new board states resulting from each move.
def all_moves(self, player):
    capture_moves = []
    non_capture_moves = []
    for r in range(8):
        for c in range(8):
            if self.board[r][c].lower() == player:
                moves = self.generate_moves(r, c)
                for move in moves:
                    if move[2] is not None:
                        capture_moves.append(((r, c), move))
                    else:
                        non_capture_moves.append(((r, c), move))
                        #mandatory capture rule
    moves = capture_moves if capture_moves else non_capture_moves
    boards = []
    for (r, c), move in moves:
        new_board = deep_copy_board(self.board)
        piece = new_board[r][c]
        new_board[r][c] = "."
        dest_r, dest_c, capture = move
        # Promote pawn to king if it reaches the last row.
        if capture is not None:
            cap_r, cap_c = capture
            new_board[cap_r][cap_c] = "."
        if piece == "b" and dest_r == 7:
            piece = "B"
        elif piece == "w" and dest_r == 0:
            piece = "W"
        new_board[dest_r][dest_c] = piece
        new_history = self.history + [self.board]
        boards.append(CurrentBoard(new_board, history=new_history))
    return boards


In [221]:
def evaluate(self):
    ''' Base Evaluation (Material, Mobility, Static Center) '''
    base_score = 0 #values of pieces
    mobility_white = 0 #total num of legal moves
    mobility_black = 0
    static_center_bonus = 0 #reward more central pieces
    forced_capture_bonus = 0 # encourages positions where pieces are forced to capture
    white_forced_capture = False #a flag that becomes True if any White piece has a capture move available
    black_forced_capture = False
    center_squares = {(3, 3), (3, 4), (4, 3), (4, 4)}
    for r in range(8):
        for c in range(8):
            cell = self.board[r][c]
            if cell == "W":
                base_score += 2
                if (r, c) in center_squares:
                    static_center_bonus += 0.5
                moves = self.generate_moves(r, c)
                mobility_white += len(moves)
                if any(move[2] is not None for move in moves):
                    white_forced_capture = True
            elif cell == "w":
                base_score += 1
                if (r, c) in center_squares:
                    static_center_bonus += 0.3
                moves = self.generate_moves(r, c)
                mobility_white += len(moves)
                if any(move[2] is not None for move in moves):
                    white_forced_capture = True
            elif cell == "B":
                base_score -= 2
                if (r, c) in center_squares:
                    static_center_bonus -= 0.5
                moves = self.generate_moves(r, c)
                mobility_black += len(moves)
                if any(move[2] is not None for move in moves):
                    black_forced_capture = True
            elif cell == "b":
                base_score -= 1
                if (r, c) in center_squares:
                    static_center_bonus -= 0.3
                moves = self.generate_moves(r, c)
                mobility_black += len(moves)
                if any(move[2] is not None for move in moves):
                    black_forced_capture = True
    mobility_score = 0.1 * (mobility_white - mobility_black)
    if white_forced_capture:
        forced_capture_bonus += 0.5
    if black_forced_capture:
        forced_capture_bonus -= 0.5
    total_score = base_score + static_center_bonus + mobility_score + forced_capture_bonus

    '''Penalise repeating positions '''
    for hist_board in self.history:
        if self.board == hist_board:
            total_score -= 200
            break
    if self.history and self.board == self.history[-1]:
        total_score -= 500
    rep_count = sum(1 for hist_board in self.history if hist_board == self.board)
    if rep_count > 1:
        total_score -= 300 * (rep_count - 1)
    if self.history:
        prev_white = sum(cell in ("w", "W") for row in self.history[-1] for cell in row)
        prev_black = sum(cell in ("b", "B") for row in self.history[-1] for cell in row)
        cur_white = sum(cell in ("w", "W") for row in self.board for cell in row)
        cur_black = sum(cell in ("b", "B") for row in self.board for cell in row)
        if cur_white == prev_white and cur_black == prev_black:
            total_score -= 100

    '''Bonus for material improvement from previous move'''
    if self.history:
        prev_board = self.history[-1]
        prev_white = sum(cell in ("w", "W") for row in prev_board for cell in row)
        prev_black = sum(cell in ("b", "B") for row in prev_board for cell in row)
        cur_white = sum(cell in ("w", "W") for row in self.board for cell in row)
        cur_black = sum(cell in ("b", "B") for row in self.board for cell in row)
        total_score += ((prev_black - cur_black) - (prev_white - cur_white)) * 15

    '''Bonus for having pieces near the center'''
    dynamic_center_bonus = 0
    center_x, center_y = 3.5, 3.5
    for r in range(8):
        for c in range(8):
            if self.board[r][c] in ("w", "W"):
                dist = abs(r - center_x) + abs(c - center_y)
                dynamic_center_bonus += max(0, 5 - dist) * 0.2
            elif self.board[r][c] in ("b", "B"):
                dist = abs(r - center_x) + abs(c - center_y)
                dynamic_center_bonus -= max(0, 5 - dist) * 0.2
    total_score += dynamic_center_bonus

    '''Bonus for moving toward promotion and being near promotion'''
    advancement_bonus = 0
    for r in range(8):
        for c in range(8):
            if self.board[r][c] == "w":
                advancement_bonus += (7 - r) * 0.5
            elif self.board[r][c] == "b":
                advancement_bonus += r * 0.5
    promotion_bonus = 0
    for c in range(8):
        if self.board[1][c] == "w":
            promotion_bonus += 2
        if self.board[6][c] == "b":
            promotion_bonus += 2
    total_score += (advancement_bonus + promotion_bonus)

    ''' Pawn Activity Bonus for Black '''
    pawn_activity_bonus = 0
    for r in range(8):
        for c in range(8):
            if self.board[r][c] == "b":
                pawn_activity_bonus += 10
    total_score += pawn_activity_bonus

    ''' Pawn Center Bonus for Black'''
    pawn_center_bonus = 0
    for r in range(8):
        for c in range(8):
            if self.board[r][c] == "b":
                dist = abs(r - center_x) + abs(c - center_y)
                pawn_center_bonus += max(0, 7 - dist) * 2
    total_score += pawn_center_bonus

    '''Penalty for pieces on the edge'''
    edge_penalty = 0
    for r in range(8):
        for c in range(8):
            if r in (0, 7) or c in (0, 7):
                if self.board[r][c] in ("w", "W", "b", "B"):
                    edge_penalty -= 5
    total_score += edge_penalty

    ''' Aggressive Capture Bonus '''
    capture_bonus = 0
    for r in range(8):
        for c in range(8):
            piece = self.board[r][c]
            moves = self.generate_moves(r, c)
            if piece.lower() == "w":
                for move in moves:
                    if move[2] is not None:
                        capture_bonus += 50
            elif piece.lower() == "b":
                for move in moves:
                    if move[2] is not None:
                        capture_bonus -= 30
    total_score += capture_bonus

    ''' Black King Repetition Penalty'''
    if self.history:
        prev_board = self.history[-1]
        for r in range(8):
            for c in range(8):
                if self.board[r][c] == "B" and prev_board[r][c] == "B":
                    total_score -= 100

    '''If one side has no pieces, return an extreme value.'''
    cur_white = sum(cell in ("w", "W") for row in self.board for cell in row)
    cur_black = sum(cell in ("b", "B") for row in self.board for cell in row)
    if cur_white == 0:
        return -1000
    if cur_black == 0:
        return 1000

    return total_score

CurrentBoard.evaluate = evaluate


In [222]:
def state_of_board(self):

    white_moves = len(self.all_moves("w"))
    black_moves = len(self.all_moves("b"))
    if white_moves == 0:
        return "black"
    if black_moves == 0:
        return "white"
    return "Unfinished"

CurrentBoard.state_of_board = state_of_board


In [223]:
# Create an instance of CurrentBoard and display it.
cb = CurrentBoard()
cb.display()



# Check the evaluation score.
cb.evaluate()


   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  W   .   .   .  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  



11.100000000000001

In [224]:
cb = CurrentBoard()
cb.display()
CurrentBoard.all_moves = all_moves

print("State of Board:", cb.state_of_board())
print("Evaluation Score:", cb.evaluate())


   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  W   .   .   .  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  

State of Board: Unfinished
Evaluation Score: 11.100000000000001


In [225]:
cb = CurrentBoard()
cb.display()
print("Legal moves for Black:")
for new_state in cb.all_moves("b"):
    new_state.display()


   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  W   .   .   .  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  

Legal moves for Black:
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   b   .  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   .   b  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  W   B   .   .  
4    .   .   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  W   .   B   .  
4    .   .   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  W   .   .   .  
4    .   .   .   .
5  

In [226]:
cb = CurrentBoard()
cb.display()
print("Legal moves for White:")
for new_state in cb.all_moves("w"):
    new_state.display()


   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  W   .   .   .  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  

Legal moves for White:
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    W   .   b   .
3  .   .   .   .  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  .   .   .   .  
4    W   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  



In [227]:
def other(player):
    return "w" if player == "b" else "b"

#search tree
class SearchTreeNode:
    def __init__(self, board_instance, playing_as, ply=0, max_depth=20, verbose=False, verbose_threshold=3):
        self.children = [] #possible next moves
        self.value = None
        self.ply_depth = ply #tracks how many moves deep we are in the search
        self.current_board = board_instance
        self.move_for = playing_as
        self.max_depth = max_depth #the limit to how far ahead the tree will search
        self.verbose = verbose #control debugging output
        self.verbose_threshold = verbose_threshold

        # check to see reached maximum depth or if the game is over.
        state = self.current_board.state_of_board()
        if self.ply_depth >= self.max_depth or state != "Unfinished":
            if state != "Unfinished":
                self.value = 1000 if state == "white" else -1000
            else:
                self.value = self.current_board.evaluate()
        else:
            self.generate_children()

    def generate_children(self):
        possible_boards = self.current_board.all_moves(self.move_for)
        # Filter out moves that would simply revert to the immediate previous board state
        if self.current_board.history:
            parent_board = self.current_board.history[-1]
            possible_boards = [b for b in possible_boards if b.board != parent_board]
        if not possible_boards:
            self.value = self.current_board.evaluate()
        else:
          # Create a new SearchTreeNode for each possible move
            for board in possible_boards:
                next_player = other(self.move_for)
                child = SearchTreeNode(board, next_player, ply=self.ply_depth+1, max_depth=self.max_depth, verbose=self.verbose, verbose_threshold=self.verbose_threshold)
                self.children.append(child)
                 # Sort children so that the most promising moves are examined first.
            self.children.sort(key=lambda child: child.current_board.evaluate(), reverse=(self.move_for=="w"))

        # Implements the alpha-beta pruning algorithm.
        # This recursively evaluates children nodes while cutting back branches that won't affect the outcome
    def alpha_beta(self, alpha=-float("inf"), beta=float("inf")):
        if self.value is not None:
            return self.value
        if self.move_for == "w":
            value = -float("inf")
            for child in self.children:
                value = max(value, child.alpha_beta(alpha, beta))
                alpha = max(alpha, value)
                if beta <= alpha:
                    break
            self.value = value
        else:
            value = float("inf")
            for child in self.children:
                value = min(value, child.alpha_beta(alpha, beta))
                beta = min(beta, value)
                if beta <= alpha:
                    break
            self.value = value
        return self.value


In [228]:
def play_computer_vs_computer():
    current_board = CurrentBoard()
    player1 = "w"
    player2 = "b"
    current_turn = player1
    move_count = 0

    #runs until the game is over or move limit is reached.
    while current_board.state_of_board() == "Unfinished" and move_count < 50:
        print(f"\nMove {move_count}, turn: {current_turn}")
        current_board.display()
        # Build the search tree from the current board state with a given maximum depth.
        search_tree = SearchTreeNode(current_board, current_turn, max_depth=6, verbose=False)
        best_value = search_tree.alpha_beta()

        best_child = None
        for child in search_tree.children:
            if child.value == best_value:
                best_child = child
                break
        if best_child is None:
            break

        # Update the board to the best move and switch the turn.
        current_board = best_child.current_board
        current_turn = player2 if current_turn == player1 else player1
        move_count += 1

     #final board and outcome.
    current_board.display()
    final_state = current_board.state_of_board()
    if final_state == "white":
        print("White wins!")
    elif final_state == "black":
        print("Black wins!")
    else:
        print("Game over!")


In [229]:
if __name__ == "__main__":
    play_computer_vs_computer()



Move 0, turn: w
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   b   .
3  W   .   .   .  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  


Move 1, turn: b
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    W   .   b   .
3  .   .   .   .  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  


Move 2, turn: w
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    W   .   .   .
3  .   .   .   b  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  


Move 3, turn: b
   a b c d e f g h
0    .   .   .   .
1  .   W   .   .  
2    .   .   .   .
3  .   .   .   b  
4    .   B   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  


Move 4, turn: w
   a b c d e f g h
0    .   .   .   .
1  .   W   .   .  
2    .   .   .   .
3  .   B   .   b  
4    .   .   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  


Move 5, turn: b
   a b c d e f g h
0    W   .   .   .
